In [1]:
#from workalendar.usa import california
import pandas as pd

In [2]:
def make_test_df():

    df = pd.DataFrame(
        {'service_date':pd.date_range(start = "2022-05-15",
                                      end = "2022-10-15")})
    
    df = df.assign(
        service_date = pd.to_datetime(df.service_date)
    )
    df = df.assign(
        day_name = df.service_date.dt.day_name(),
        day_of_week = df.service_date.dt.dayofweek,
        month = df.service_date.dt.month, 
        year = df.service_date.dt.year,
        quarter = df.service_date.dt.quarter,
    )
    
    df = df.assign(
        weekday = df.apply(lambda x: 1 if x.day_of_week <= 4 else 0, axis=1)
    )
    
    return df

In [4]:
df = make_test_df()
holidays = pd.read_parquet("./holidays.parquet")

In [7]:
df2 = pd.merge(
    df, 
    holidays.rename(columns = {"date": "service_date"})[["service_date", "holiday"]],
    on = ["service_date"],
    how = "left",
)

df2 = df2.assign(
    weekday = df2.apply(lambda x: 0 if x.holiday==1 else x.weekday, axis=1),
    holiday = df2.holiday.fillna(0).astype(int)
)

In [9]:
df2[df2.month==9].head()

,service_date,day_name,day_of_week,month,year,quarter,weekday,holiday
109,2022-09-01,Thursday,3,9,2022,3,1,0
110,2022-09-02,Friday,4,9,2022,3,1,0
111,2022-09-03,Saturday,5,9,2022,3,0,0
112,2022-09-04,Sunday,6,9,2022,3,0,0
113,2022-09-05,Monday,0,9,2022,3,0,1


In [ ]:
# https://stackoverflow.com/questions/13145368/how-to-find-the-maximum-value-in-a-list-of-tuples
#from operator import itemgetter

#max_date = max(CA_HOLIDAYS,key=itemgetter(0))[0]   #faster solution
#itemgetter(0)